In [1]:
# load data
import numpy as np
train_file = np.load('data/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)
test_file = np.load('data/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


In [2]:
# preprocess
def preprocess_data(data):
    """
    Removes padded agents (agents with all zero values across time steps).
    
    Args:
        data (numpy.ndarray): Shape (scenarios, agents, time_steps, dimensions)
    
    Returns:
        numpy.ndarray: Filtered dataset without padded agents.
    """
    scenarios, agents, time_steps, dimensions = data.shape
    processed_data = []

    for i in range(scenarios):
        scenario_data = data[i]  # Shape (agents, time_steps, dimensions)
        
        # Identify non-padded agents (at least one nonzero value across all time steps)
        valid_agents = np.any(scenario_data != 0, axis=(1, 2))  # Shape (agents,)
        
        # Filter out only the valid agents
        filtered_agents = scenario_data[valid_agents]  # Shape (valid_agents, time_steps, dimensions)
        
        processed_data.append(filtered_agents)

    return processed_data  # List of variable-length arrays per scenario

train_data_processed = preprocess_data(train_data)
test_data_processed = preprocess_data(test_data)

# Print results
print(f"Original Train Data Shape: {train_data.shape}")
print(f"Processed Train Data Length: {len(train_data_processed)} (variable agents per scenario)")

print(f"Original Test Data Shape: {test_data.shape}")
print(f"Processed Test Data Length: {len(test_data_processed)} (variable agents per scenario)")

Original Train Data Shape: (10000, 50, 110, 6)
Processed Train Data Length: 10000 (variable agents per scenario)
Original Test Data Shape: (2100, 50, 50, 6)
Processed Test Data Length: 2100 (variable agents per scenario)


In [3]:
# some kind of missing trajectory handling? 

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed, Dropout

/Users/mehul/Documents/UCSD Classes/Deep Learning/AgroverseComp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
import tensorflow as tf

def angle_change_loss(y_pred):
    """
    Penalize large changes in direction between consecutive deltas.

    Args:
        y_pred: Tensor of shape (batch_size, Tpred, 2)

    Returns:
        Scalar loss penalizing angle differences
    """
    # Normalize deltas to unit vectors
    delta_unit = tf.math.l2_normalize(y_pred, axis=-1)  # shape: (B, T, 2)

    # Compute cosine similarity between consecutive deltas
    dot_products = tf.reduce_sum(delta_unit[:, 1:, :] * delta_unit[:, :-1, :], axis=-1)  # shape: (B, T-1)

    # Clamp for numerical stability (to avoid NaNs in arccos)
    dot_products = tf.clip_by_value(dot_products, -1.0, 1.0)

    # Compute angle in radians between -1 and 1 (cos⁻¹)
    angle_diff = tf.acos(dot_products)  # shape: (B, T-1)

    # Mean angle difference per sequence
    return tf.reduce_mean(angle_diff)

def combined_loss(y_true, y_pred):
    # Check if there are NaN values in y_true or y_pred
    nan_check = tf.reduce_any(tf.math.is_nan(y_true)) | tf.reduce_any(tf.math.is_nan(y_pred))

    # Use tf.cond to perform the check
    def return_nan_loss():
        return tf.constant(float('nan'))

    def calculate_loss():
        mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
        angle_loss = angle_change_loss(y_pred)
        return mse_loss + 0.5 * angle_loss

    return tf.cond(nan_check, return_nan_loss, calculate_loss)




In [62]:
def create_lstm_encoder_decoder(input_dim, output_dim, timesteps_in, timesteps_out,
                                lstm_units=512, num_layers=3, loss_fn='mse', lr=0.001):
    inputs = Input(shape=(timesteps_in, input_dim))

    # Encoder
    x = inputs
    for _ in range(num_layers):
        x = LSTM(lstm_units, return_sequences=True)(x)
        # x = Dropout(0.2)(x)
    encoded = LSTM(lstm_units)(x)  # Final encoder output

    # Decoder
    x = RepeatVector(timesteps_out)(encoded)
    for _ in range(num_layers):
        x = LSTM(lstm_units, return_sequences=True)(x)
        # x = Dropout(0.2)(x)
    
    
    x = TimeDistributed(Dense(128, activation='relu'))(x)
    x = TimeDistributed(Dense(64, activation='relu'))(x)
    outputs = TimeDistributed(Dense(output_dim))(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse', metrics=['mae']) 

    # model.compile(optimizer=Adam(learning_rate=1e-5), loss=combined_loss) #switch loss back to 'mse' for older solution 
    return model

In [72]:
from keras.src.callbacks import LearningRateScheduler, EarlyStopping, Callback
from keras.src.optimizers import Adam
from keras import Model
import numpy as np


def exponential_decay_schedule(epoch, lr):
    decay_rate = 0.9
    decay_steps = 5
    if epoch % decay_steps == 0 and epoch:
        print('Learning rate update:', lr * decay_rate)
        return lr * decay_rate
    return lr


# Custom callback to monitor LR and stop training
class LRThresholdCallback(Callback):
    def __init__(self, threshold=9e-5):
        super().__init__()
        self.threshold = threshold
        self.should_stop = False

    def on_epoch_end(self, epoch, logs=None):
        lr = float(self.model.optimizer.learning_rate.numpy())
        if lr < self.threshold:
            print(f"\nLearning rate {lr:.6f} < threshold {self.threshold}, moving to Phase 2.")
            self.model.stop_training = True

def train_model(train_data, batch_size=32, validation_split=0.2, Tobs=50, Tpred=60):
    n_scenarios = train_data.shape[0]
    X_train_raw = []
    y_train_deltas = []

    for i in range(n_scenarios):
        ego_data = train_data[i, 0, :, :]
        if np.all(ego_data == 0):
            continue

        observed = ego_data[:Tobs]            # shape (50, 6)
        future = ego_data[Tobs:Tobs+Tpred, :2]
        last_obs_pos = observed[-1, :2]

        if np.any(np.all(observed == 0, axis=1)) or np.any(np.all(future == 0, axis=1)):
            continue

        # Compute deltas w.r.t. previous future timestep
        delta = np.diff(np.vstack([last_obs_pos, future]), axis=0)  # (60, 2)

        X_train_raw.append(observed)
        y_train_deltas.append(delta)
    

    X_train = np.array(X_train_raw)
    y_train = np.array(y_train_deltas)

    print(f"Training on {X_train.shape[0]} valid sequences.")
    print(f"Input shape: {X_train.shape}, Delta Output shape: {y_train.shape}")
    
    # --- Normalize Input and Output ---
    X_mean = X_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 6)
    X_std = X_train.std(axis=(0, 1), keepdims=True) + 1e-8

    y_mean = y_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 2)
    y_std = y_train.std(axis=(0, 1), keepdims=True) + 1e-8

    X_train = (X_train - X_mean) / X_std
    y_train = (y_train - y_mean) / y_std
    
    print(X_train[:2])
    print(y_train[:2])

    model = create_lstm_encoder_decoder(
        input_dim=X_train.shape[-1],
        output_dim=2,
        timesteps_in=Tobs,
        timesteps_out=Tpred,
        loss_fn='mse',
        lr=0.001
    )

    phase1_callbacks = [
        LearningRateScheduler(exponential_decay_schedule),
        EarlyStopping(patience=4, restore_best_weights=True, monitor='val_loss'),
        LRThresholdCallback(threshold=9e-5)
    ]

    print("\n--- Phase 1: Training ---")
    model.fit(
        X_train, y_train,
        epochs=500,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=phase1_callbacks,
        verbose=1
    )

    print("\n--- Phase 2: Fine-tuning ---")
    model.compile(optimizer=Adam(1e-4), loss='mse', metrics=['mae'])
    phase2_callbacks = [
        LearningRateScheduler(exponential_decay_schedule),
        EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')
    ]
    model.fit(
        X_train, y_train,
        epochs=500,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=phase2_callbacks,
        verbose=1
    )

    # Return model and normalization parameters
    return model, X_mean, X_std, y_mean, y_std


In [73]:
import pickle

def save_model(model, filepath='lstm_1.pkl'):
    """Save model and scaler together in a pickle file"""
    model_json = model.to_json()
    model_weights = model.get_weights()
    data = {
        'model_json': model_json,
        'model_weights': model_weights,
    }
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    print(f"Model saved to {filepath}")

def load_model(filepath='lstm_1.pkl'):
    """Load model and scaler from pickle file"""
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    
    # Reconstruct model
    model = tf.keras.models.model_from_json(data['model_json'])
    model.set_weights(data['model_weights'])
    model.compile(optimizer='adam', loss='mse')
    
    return model

In [71]:
# first check that the model can overfit on small data
model, X_mean, X_std, y_mean, y_std  = train_model(train_data[:20], batch_size=20, validation_split=0)

Process SpawnProcess-15:
Process SpawnProcess-12:
Process SpawnProcess-16:
Process SpawnProcess-10:
Process SpawnProcess-14:
Process SpawnProcess-13:
Process SpawnProcess-11:
Process SpawnProcess-9:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 102, in get
    with self._rlock:
  F

KeyboardInterrupt: 

In [74]:
def plot_mae_by_timestep(y_true, y_pred):
    """
    Visualize MAE across timesteps in the prediction horizon.
    
    Args:
        y_true (np.ndarray): shape (N, Tpred, 2)
        y_pred (np.ndarray): shape (N, Tpred, 2)
    """
    mae_per_timestep = np.mean(np.abs(y_true - y_pred), axis=(0, 2))  # shape (Tpred,)
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))
    plt.plot(mae_per_timestep, label='MAE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('MAE (meters)')
    plt.title('Mean Absolute Error Over Prediction Horizon')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [75]:
def reconstruct_absolute_positions(pred_deltas, last_observed_positions):
    """
    Reconstruct absolute predicted positions by adding deltas to the last observed position.

    Args:
        pred_deltas: np.ndarray of shape (N, Tpred, 2)
        last_observed_positions: np.ndarray of shape (N, 2)

    Returns:
        np.ndarray of shape (N, Tpred, 2)
    """
    return last_observed_positions[:, None, :] + np.cumsum(pred_deltas, axis=1)



def forecast_positions(scenario_data, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std):
    """
    Use normalized LSTM model to forecast future deltas and reconstruct absolute positions.

    Args:
        scenario_data (numpy.ndarray): Shape (agents, time_steps, dimensions)
        Tobs (int): Number of observed time steps
        Tpred (int): Number of future time steps to predict
        model (Model): Trained LSTM model that predicts normalized deltas
        X_mean, X_std: Normalization stats for input
        y_mean, y_std: Normalization stats for output

    Returns:
        numpy.ndarray: Predicted absolute positions of shape (agents, Tpred, 2)
    """
    agents, _, _ = scenario_data.shape
    predicted_positions = np.zeros((agents, Tpred, 2))
    pred_deltas_all = []

    for agent_idx in range(agents):
        agent_data = scenario_data[agent_idx, :Tobs, :]  # shape (Tobs, 6)

        # Skip if fully padded
        if np.all(agent_data == 0):
            continue

        # Normalize input
        X_pred = np.expand_dims(agent_data, axis=0)  # shape (1, Tobs, 6)
        X_pred_norm = (X_pred - X_mean) / X_std

        # Predict normalized deltas
        pred_deltas_norm = model.predict(X_pred_norm, verbose=0)  # shape (1, Tpred, 2)

        # Denormalize deltas
        pred_deltas = pred_deltas_norm * y_std + y_mean
        pred_deltas_all.append(pred_deltas[0])

        # Reconstruct absolute positions
        last_pos = agent_data[Tobs - 1, :2]  # shape (2,)
        abs_positions = reconstruct_absolute_positions(
            pred_deltas=pred_deltas,
            last_observed_positions=np.expand_dims(last_pos, axis=0)
        )[0]

        predicted_positions[agent_idx] = abs_positions

    return predicted_positions


In [76]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def make_gif(data_matrix1, data_matrix2, name='comparison'):
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    cmap1 = plt.cm.get_cmap('viridis', 50)
    cmap2 = plt.cm.get_cmap('plasma', 50)

    assert data_matrix1.shape[1] == data_matrix2.shape[1], "Both matrices must have same number of timesteps"
    timesteps = data_matrix1.shape[1]

    fig, axes = plt.subplots(1, 2, figsize=(18, 9))
    ax1, ax2 = axes

    def update(frame):
        for ax in axes:
            ax.clear()

        for i in range(data_matrix1.shape[0]):
            for (data_matrix, ax, cmap) in [(data_matrix1, ax1, cmap1), (data_matrix2, ax2, cmap2)]:
                x = data_matrix[i, frame, 0]
                y = data_matrix[i, frame, 1]
                if x != 0 and y != 0:
                    xs = data_matrix[i, :frame+1, 0]
                    ys = data_matrix[i, :frame+1, 1]
                    mask = (xs != 0) & (ys != 0)
                    xs = xs[mask]
                    ys = ys[mask]
                    if len(xs) > 0 and len(ys) > 0:
                        color = cmap(i)
                        ax.plot(xs, ys, alpha=0.9, color=color)
                        ax.scatter(x, y, s=80, color=color)

        # Plot ego vehicle (index 0) on both
        ax1.plot(data_matrix1[0, :frame, 0], data_matrix1[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
        ax1.scatter(data_matrix1[0, frame, 0], data_matrix1[0, frame, 1], s=80, color='tab:orange')
        ax1.set_title('Prediction')

        ax2.plot(data_matrix2[0, :frame, 0], data_matrix2[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
        ax2.scatter(data_matrix2[0, frame, 0], data_matrix2[0, frame, 1], s=80, color='tab:orange')
        ax2.set_title('Actual')

        for ax, data_matrix in zip(axes, [data_matrix1, data_matrix2]):
            ax.set_xlim(data_matrix[:, :, 0][data_matrix[:, :, 0] != 0].min() - 10,
                        data_matrix[:, :, 0][data_matrix[:, :, 0] != 0].max() + 10)
            ax.set_ylim(data_matrix[:, :, 1][data_matrix[:, :, 1] != 0].min() - 10,
                        data_matrix[:, :, 1][data_matrix[:, :, 1] != 0].max() + 10)
            ax.legend()
            ax.set_xlabel('X')
            ax.set_ylabel('Y')

        # Compute MSE over non-zero entries up to current frame
        mask = (data_matrix2[:, :frame+1, :] != 0) & (data_matrix1[:, :frame+1, :] != 0)
        mse = np.mean((data_matrix1[:, :frame+1, :][mask] - data_matrix2[:, :frame+1, :][mask]) ** 2)

        fig.suptitle(f"Timestep {frame} - MSE: {mse:.4f}", fontsize=16)
        return ax1.collections + ax1.lines + ax2.collections + ax2.lines

    anim = animation.FuncAnimation(fig, update, frames=list(range(0, timesteps, 3)), interval=100, blit=True)
    anim.save(f'trajectory_visualization_{name}.gif', writer='pillow')
    plt.close()


In [ ]:
# visualize prediction

# model = load_model()

# Parameters
Tobs = 50
Tpred = 60

data = train_data[250]

# Select a test scenario (can use any valid index)
test_scenario = data.copy()  # shape (agents, time_steps, features)

# Forecast future positions
predicted_positions = forecast_positions(test_scenario, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std)

# Create combined matrix of past observed + predicted for ego agent (agent 0)
ego_past = test_scenario[0, :Tobs, :2]               # shape (Tobs, 2)
ego_future = predicted_positions[0]                  # shape (Tpred, 2)
ego_full = np.concatenate([ego_past, ego_future], axis=0)  # shape (Tobs + Tpred, 2)

# Create updated scenario with predicted ego and original others
updated_scenario = test_scenario.copy()
updated_scenario[0, :Tobs+Tpred, :2] = ego_full  # Replace ego trajectory

# Visualize
make_gif(updated_scenario, data, name='lstm1')


In [77]:
# Train the model
model, X_mean, X_std, y_mean, y_std = train_model(train_data)

# Save the model 
save_model(model)

Training on 10000 valid sequences.
Input shape: (10000, 50, 6), Delta Output shape: (10000, 60, 2)
[[[-0.70963494 -0.99812224 -0.9788003   0.3573056   1.55063073
    0.        ]
  [-0.70979287 -0.99802844 -0.9788003   0.3573056   1.55055584
    0.        ]
  [-0.70998773 -0.99791262 -1.98200353  0.70357905  1.55046169
    0.        ]
  [-0.71021821 -0.99777552 -1.97045983  0.70338832  1.55035232
    0.        ]
  [-0.71048173 -0.9976186  -1.9682069   0.70515924  1.55023716
    0.        ]
  [-0.710774   -0.99744439 -1.97748584  0.70712759  1.55012029
    0.        ]
  [-0.71108946 -0.99725619 -1.95658731  0.69991828  1.54999826
    0.        ]
  [-0.71142111 -0.99705816 -1.9576821   0.70039375  1.54987536
    0.        ]
  [-0.71176055 -0.99685533 -1.97952503  0.70802667  1.54975052
    0.        ]
  [-0.71209799 -0.99665355 -1.9673489   0.70639174  1.54962596
    0.        ]
  [-0.71242228 -0.99645941 -1.95530018  0.70477391  1.54951852
    0.        ]
  [-0.71273701 -0.99627079 -1.97

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error


def evaluate_mse(train_data, model, Tobs=50, Tpred=60):
    """
    Computes LSTM prediction for ego agent and evaluates MSE with progress reporting.
    """
    N = train_data.shape[0]
    mse_list = []
    valid_scenarios = 0
    
    print(f"Evaluating {N} scenarios...")
    
    # Progress reporting variables
    report_interval = max(1, N // 10)  # Report at 10% intervals
    
    for i in range(N):
        # Progress reporting
        if i % report_interval == 0 or i == N-1:
            print(f"Processing scenario {i+1}/{N} ({(i+1)/N*100:.1f}%)")
        
        scenario_data = train_data[i]
        ego_agent_data = scenario_data[0]
        ground_truth = ego_agent_data[Tobs:Tobs+Tpred, :2]
        
        # Skip if ground truth contains all zeros (padded)
        if np.all(ground_truth == 0):
            continue
            
        valid_scenarios += 1
        
        # Forecast future positions
        predicted_positions = forecast_positions(
            ego_agent_data[np.newaxis, :, :],
            Tobs, Tpred, model, X_mean, X_std, y_mean, y_std
        )
        
        # Compute MSE
        mse = mean_squared_error(ground_truth, predicted_positions[0])
        mse_list.append(mse)
        
        # Occasional MSE reporting
        if i % report_interval == 0:
            print(f"  Current scenario MSE: {mse:.4f}")
    
    # Final results
    if mse_list:
        overall_mse = np.mean(mse_list)
        print(f"Evaluation complete: {valid_scenarios} valid scenarios")
        print(f"Mean Squared Error (MSE): {overall_mse:.4f}")
        print(f"Min MSE: {np.min(mse_list):.4f}, Max MSE: {np.max(mse_list):.4f}")
        return overall_mse
    else:
        print("No valid scenarios for evaluation.")
        return None

In [ ]:
# Evaluate on training data
evaluate_mse(train_data, model)